In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# path to zipped & working directories
path_zip = '/kaggle/input/denoising-dirty-documents/'
path = '/kaggle/working/'

# unzip files first to working directory
# We could use also unzipped data source, but why not to learn something new?
with zipfile.ZipFile(path_zip + 'train.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

with zipfile.ZipFile(path_zip + 'test.zip', 'r') as zip_ref:
    zip_ref.extractall(path)  
    
with zipfile.ZipFile(path_zip + 'train_cleaned.zip', 'r') as zip_ref:
    zip_ref.extractall(path)  
    
with zipfile.ZipFile(path_zip + 'sampleSubmission.csv.zip', 'r') as zip_ref:
    zip_ref.extractall(path)  

In [ ]:
# store image names in list for later use
train_img = sorted(os.listdir(path + '/train'))
train_cleaned_img = sorted(os.listdir(path + '/train_cleaned'))
test_img = sorted(os.listdir(path + '/test'))

In [ ]:
# prepare function
dataset_path = "/kaggle/working/train"
def process_image(path):
    img = cv2.imread(path)
    img = np.asarray(img, dtype="float32")
    img = cv2.resize(img, (540, 420))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255.0
    img = np.reshape(img,(420, 540, 1))
    
    return img

In [ ]:
# preprocess images
train = []
train_cleaned = []
test = []

for f in sorted(os.listdir(path + 'train/')):
    train.append(process_image(path + 'train/' + f))

for f in sorted(os.listdir(path + 'train_cleaned/')):
    train_cleaned.append(process_image(path + 'train_cleaned/' + f))
   
for f in sorted(os.listdir(path + 'test/')):
    test.append(process_image(path + 'test/' + f))

In [ ]:
plt.figure(figsize=(15,25))
for i in range(0,8,2):
    plt.subplot(4,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train[i][:,:,0], cmap='gray')
    plt.title('Noise image: {}'.format(train_img[i]))
    
    plt.subplot(4,2,i+2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_cleaned[i][:,:,0], cmap='gray')
    plt.title('Denoised image: {}'.format(train_img[i]))

plt.show()

In [ ]:
# convert list to numpy array
X_train = np.asarray(train)
y_train = np.asarray(train_cleaned)
X_test = np.asarray(test)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)

In [ ]:
conv_autoencoder = Sequential()

# Encoder
conv_autoencoder.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(420,540,1), activation='relu', padding='same'))
conv_autoencoder.add(MaxPooling2D((2, 2), padding='same'))

conv_autoencoder.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same'))
conv_autoencoder.add(MaxPooling2D((2, 2), padding='same'))



# Decoder
conv_autoencoder.add(Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'))
conv_autoencoder.add(Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'))


# Output
conv_autoencoder.add(Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same'))

conv_autoencoder.summary()

In [ ]:
pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [ ]:
wandb.login()

In [ ]:
import wandb
from tensorflow.keras.callbacks import EarlyStopping
from wandb.keras import WandbCallback

# Initialize WandB
wandb.init(project='task1-conv-autoencoder', name='Image_Pre')

# Compile the model
conv_autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Define the early stopping callback
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Train the model
history = conv_autoencoder.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=16, callbacks=[early_stop, WandbCallback()])

# Save the model architecture as JSON
model_json = conv_autoencoder.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Log the model architecture
wandb.save("model_architecture.json")

# Log the model summary
conv_autoencoder.summary(print_fn=lambda x: wandb.log({"model_summary": x}))

# Finish the run
wandb.run.finish()

In [ ]:
# Check how loss & mae went down
epoch_loss = history.history['loss']
epoch_val_loss = history.history['val_loss']
epoch_mae = history.history['mse']
epoch_val_mae = history.history['val_mse']

plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
plt.plot(range(0,len(epoch_loss)), epoch_loss, 'b-', linewidth=2, label='Train Loss')
plt.plot(range(0,len(epoch_val_loss)), epoch_val_loss, 'r-', linewidth=2, label='Val Loss')
plt.title('Evolution of loss on train & validation datasets over epochs')
plt.legend(loc='best')

plt.subplot(1,2,2)
plt.plot(range(0,len(epoch_mae)), epoch_mae, 'b-', linewidth=2, label='Train MSE')
plt.plot(range(0,len(epoch_val_mae)), epoch_val_mae, 'r-', linewidth=2,label='Val MSE')
plt.title('Evolution of MSE on train & validation datasets over epochs')
plt.legend(loc='best')

plt.show()

In [ ]:
y_pred = conv_autoencoder.predict(X_test, batch_size=16)

In [ ]:
import time

# Start the timer
start_time = time.time()

# Your task code here
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.xticks([])
plt.yticks([])
plt.imshow(X_test[0][:, :, 0], cmap='gray')
plt.title('Noisy Image: {}'.format(test_img[0]))

plt.subplot(1, 2, 2)
plt.xticks([])
plt.yticks([])
plt.imshow(y_pred[0][:, :, 0], cmap='gray')
plt.title('Denoised Image: {}'.format(test_img[0]))

plt.tight_layout()
plt.show()

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Time taken: {:.2f} seconds".format(elapsed_time))


In [ ]:
plt.figure(figsize=(15,25))
for i in range(0,8,2):
    plt.subplot(4,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_test[i][:,:,0], cmap='gray')
    plt.title('Noisy image: {}'.format(test_img[i]))
    
    plt.subplot(4,2,i+2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(y_pred[i][:,:,0], cmap='gray')
    plt.title('Denoised by autoencoder: {}'.format(test_img[i]))

plt.show()

# To adapt the model to handle large images, we will implement the sliding window technique.

In [ ]:
import numpy as np
import cv2

def add_padding(image, window_size, stride):
    h, w = image.shape[:2]
    pad_h = (stride - (h - window_size[0]) % stride) % stride
    pad_w = (stride - (w - window_size[1]) % stride) % stride
    padded_image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=255)
    return padded_image


def sliding_window(image, window_size, stride):
    h, w = image.shape[:2]
    for y in range(0, h - window_size[0] + 1, stride):
        for x in range(0, w - window_size[1] + 1, stride):
            yield x, y, image[y:y + window_size[0], x:x + window_size[1]]


def test_model_on_large_image(model, large_image_path, window_size, stride):
    large_image = cv2.imread(large_image_path, cv2.IMREAD_GRAYSCALE)
    padded_image = add_padding(large_image, window_size, stride)
    
    output_image = np.zeros_like(padded_image, dtype=np.float32)  # Specify float32 data type
    count_map = np.zeros_like(padded_image)

    for x, y, window in sliding_window(padded_image, window_size, stride):
        window = np.expand_dims(window, axis=-1)
        prediction = model.predict(np.expand_dims(window,axis=0))

        output_image[y:y + window_size[0], x:x + window_size[1]] += prediction[0,:,:,0].astype(np.float32)

        count_map[y:y + window_size[0], x:x + window_size[1]] += 1

    # Clip the output_image to the range [0, 255] and convert it back to uint8
    output_image = np.clip(output_image / count_map , 0 , 255).astype(np.uint8)

    # Remove padding by cropping to the original size
    pad_h = (stride - (large_image.shape[0] - window_size[0]) % stride) % stride
    pad_w = (stride - (large_image.shape[1] - window_size[1]) % stride) % stride
    output_image = output_image[:large_image.shape[0]+pad_h,:large_image.shape[1]+pad_w]

    return output_image


In [ ]:
large_image_path="/kaggle/input/image6/6.jpeg"

In [ ]:
import cv2

# Load the image using OpenCV
image_path ="/kaggle/input/image6/6.jpeg"
image = cv2.imread(image_path)

# Get the height and width of the image
height, width = image.shape[:2]

# Print the size of the image
print(f"Image size: {width}x{height}")


In [ ]:
output_image = test_model_on_large_image(conv_autoencoder, large_image_path, target_size=(1300,1060), window_size=(420,540), stride=420)

In [ ]:
output_image = test_model_on_large_image(conv_autoencoder, large_image_path, window_size=(420,540), stride=30)

In [ ]:
original_img=large_image_path
origi=cv2.imread(original_img, cv2.IMREAD_GRAYSCALE)

In [ ]:
# Remove padding by cropping to the original size (exclude black areas at the right and bottom)
output_image = output_image[:origi.shape[0], :origi.shape[1]]

# Display the original and output images side by side with their original sizes
plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# Original image subplot
plt.subplot(1, 2, 1)  # 1 row, 2 columns, subplot index 1
plt.imshow(origi, cmap='gray', aspect='auto')  # Set aspect to 'auto' to preserve the original aspect ratio
plt.title('Original Image')
plt.axis('off')  # Turn off axes for a cleaner display

# Output image subplot
plt.subplot(1, 2, 2)  # 1 row, 2 columns, subplot index 2
plt.imshow(output_image, cmap='gray', aspect='auto')  # Set aspect to 'auto' to preserve the original aspect ratio
plt.title('SCanned Image')
plt.axis('off')  # Turn off axes for a cleaner display

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# Original image subplot
plt.subplot(1, 2, 1)  # 1 row, 2 columns, subplot index 1
plt.imshow(origi, cmap='gray')
plt.title('Original Image')
plt.axis('off')  # Turn off axes for a cleaner display

# Output image subplot
plt.subplot(1, 2, 2)  # 1 row, 2 columns, subplot index 2
plt.imshow(output_image, cmap='gray')
plt.title('Output Image')
plt.axis('off')  # Turn off axes for a cleaner display

plt.show()